In [885]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import math
import statsmodels.api as sm
from statsmodels import regression, stats
import statsmodels
import scipy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import chisquare

In [886]:
DATA_PATH = '/Users/juliusraschke/Documents/Quantitative Finance/Summer Semester 2/Advanced Quant Finance/Data'

### Load in VIX Term Structure

In [887]:
VIX_TS = pd.read_csv(DATA_PATH + '/vixts.csv',skiprows=4)

VIX_TS = VIX_TS.rename({'Date':'DATE','VIX_1^2':'1 month','VIX_2^2':'2 month','VIX_3^2':'3 month','VIX_6^2':'6 month',
               'VIX_9^2':'9 month','VIX_12^2':'12 month'},axis=1)

In [888]:
def DATE_STR(num):
    num_str = str(num)
    date_str = num_str[-2:]+'/'+num_str[4:-2]+'/'+num_str[0:4]
    
    return date_str 

VIX_TS['DATE'] = VIX_TS['DATE'].apply(DATE_STR)
VIX_TS['DATE'] = pd.to_datetime(VIX_TS['DATE'],format = '%d/%m/%Y')

### Load in VIX Future Returns

In [889]:
VIX_FUT = pd.read_csv(DATA_PATH + '/vixfut_ret.csv',skiprows=5)

VIX_FUT = VIX_FUT.rename({'Date':'DATE','r_1^{VIX fut.}':'1 month','r_2^{VIX fut.}':'2 month','r_3^{VIX fut.}':'3 month',
                         'r_4^{VIX fut.}':'4 month','r_5^{VIX fut.}':'5 month','r_6^{VIX fut.}':'6 month'},axis=1)

In [890]:
VIX_FUT['DATE'] = VIX_FUT['DATE'].apply(DATE_STR)
VIX_FUT['DATE'] = pd.to_datetime(VIX_FUT['DATE'],format = '%d/%m/%Y')

In [891]:
column_mapping = {
    '1 month': 'V_F_R 1 month',
    '2 month': 'V_F_R 2 month',
    '3 month': 'V_F_R 3 month',
    '4 month': 'V_F_R 6 month',
    '5 month': 'V_F_R 9 month',
    '6 month': 'V_F_R 12 month',
}

VIX_FUT.rename(columns=column_mapping, inplace=True)

### Load in Variance Swap Returns

In [892]:
VAR_SWAPS = pd.read_csv(DATA_PATH + '/vswap_ret.csv',skiprows=5)

VAR_SWAPS = VAR_SWAPS.rename({'Date':'DATE','r_1^{var. swap}':'1 month','r_2^{var. swap}':'2 month','r_3^{var. swap}':'3 month',
                         'r_6^{var. swap}':'6 month','r_9^{var. swap}':'9 month','r_12^{var. swap}':'12 month'},axis=1)

In [893]:
VAR_SWAPS['DATE'] = VAR_SWAPS['DATE'].apply(DATE_STR)
VAR_SWAPS['DATE'] = pd.to_datetime(VAR_SWAPS['DATE'],format = '%d/%m/%Y')

In [894]:
column_mapping = {
    '1 month': 'V_S_R 1 month',
    '2 month': 'V_S_R 2 month',
    '3 month': 'V_S_R 3 month',
    '6 month': 'V_S_R 6 month',
    '9 month': 'V_S_R 9 month',
    '12 month': 'V_S_R 12 month',
}

VAR_SWAPS.rename(columns=column_mapping, inplace=True)

### Load in Straddle Returns

In [895]:
STRADDLE = pd.read_csv(DATA_PATH + '/straddle_ret.csv',skiprows=6)

STRADDLE = STRADDLE.rename({'Date':'DATE',
                            'r_1^{straddle}':'1 month Long','r_2^{straddle}':'2 month Long','r_3^{straddle}':'3 month Long',
                            'r_6^{straddle}':'6 month Long','r_9^{straddle}':'9 month Long','r_12^{straddle}':'12 month Long',
                            'r_1^{short straddle}':'1 month Short','r_2^{short straddle}':'2 month Short','r_3^{short straddle}':'3 month Short',
                            'r_6^{short straddle}':'6 month Short','r_9^{short straddle}':'9 month Short','r_12^{short straddle}':'12 month Short',},axis=1)

In [896]:
STRADDLE['DATE'] = STRADDLE['DATE'].apply(DATE_STR)
STRADDLE['DATE'] = pd.to_datetime(STRADDLE['DATE'],format = '%d/%m/%Y')

In [897]:
STRADDLE_LONG = STRADDLE[['DATE','1 month Long','2 month Long','3 month Long','6 month Long','9 month Long','12 month Long']]

In [898]:
column_mapping = {
    '1 month Long': 'S_R 1 month',
    '2 month Long': 'S_R 2 month',
    '3 month Long': 'S_R 3 month',
    '6 month Long': 'S_R 6 month',
    '9 month Long': 'S_R 9 month',
    '12 month Long': 'S_R 12 month',
}

STRADDLE_LONG.rename(columns=column_mapping, inplace=True)

<ipython-input-898-1bc326bfab36>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STRADDLE_LONG.rename(columns=column_mapping, inplace=True)


### Load in Interest Rate Returns

In [899]:
df = pd.read_csv(DATA_PATH + '/IR.csv')

In [900]:
# Get rates for all days which are in the 300s
filtered_df = df[(df['days'] >= 300) & (df['days'] < 400)].reset_index(drop=True)

# Sort the DataFrame by 'dates' and 'days'
df_sorted = filtered_df.sort_values(by=['date', 'days'])

# Keep the rows with the closest 'days' value to 365 for each date
IR_TS = df_sorted.groupby('date', group_keys=False).apply(lambda group: group.iloc[(group['days'] - 365).abs().argsort()[:1]])

# Reset the index if needed
IR_TS.reset_index(drop=True, inplace=True)

In [901]:
IR_TS['Daily Rate'] = ((1+IR_TS['rate'])**(1/IR_TS['days']))-1
IR_TS = IR_TS.drop(['rate','days'],axis=1)

In [902]:
IR_TS.rename(columns={'date': 'DATE'}, inplace=True)

### Created Merged Databases for returns

In [903]:
IR_TS.set_index('DATE', inplace=True)
VIX_TS.set_index('DATE', inplace=True)
VAR_SWAPS.set_index('DATE', inplace=True)
VIX_FUT.set_index('DATE', inplace=True)
STRADDLE_LONG.set_index('DATE', inplace=True)

In [904]:
IR_TS.index = pd.to_datetime(IR_TS.index)
VIX_TS.index = pd.to_datetime(VIX_TS.index)
VAR_SWAPS.index = pd.to_datetime(VAR_SWAPS.index)
VIX_FUT.index = pd.to_datetime(VIX_FUT.index)
STRADDLE_LONG.index = pd.to_datetime(STRADDLE_LONG.index)

In [905]:
merged = pd.merge(VIX_TS,IR_TS, how='inner', left_index=True, right_index=True)

In [906]:
VAR_SWAPS = VAR_SWAPS/100
VIX_FUT = VIX_FUT/100
STRADDLE_LONG = STRADDLE_LONG/100

In [907]:
VAR_SWAPS_MERGED=pd.merge(merged,VAR_SWAPS, how='inner', left_index=True, right_index=True)
VIX_FUT_MERGED=pd.merge(merged,VIX_FUT, how='inner', left_index=True, right_index=True)
STRADDLE_MERGED=pd.merge(merged,STRADDLE_LONG, how='inner', left_index=True, right_index=True)

### Calculate Excess returns

In [908]:
XS_VAR_SWAPS = VAR_SWAPS_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_S_R 1 month','V_S_R 2 month', 'V_S_R 3 month', 'V_S_R 6 month', 'V_S_R 9 month', 'V_S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VAR_SWAPS[col] = XS_VAR_SWAPS[col] - VAR_SWAPS_MERGED['Daily Rate']

In [909]:
XS_VIX_FUT = VIX_FUT_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['V_F_R 1 month','V_F_R 2 month', 'V_F_R 3 month', 'V_F_R 6 month', 'V_F_R 9 month', 'V_F_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_VIX_FUT[col] = XS_VIX_FUT[col] - VIX_FUT_MERGED['Daily Rate']

In [910]:
XS_STRADDLE = STRADDLE_MERGED.iloc[:, 7:]
columns_to_subtract_from = ['S_R 1 month','S_R 2 month', 'S_R 3 month', 'S_R 6 month', 'S_R 9 month', 'S_R 12 month']

# Iterate through the columns and subtract 'column_to_subtract' from each of them
for col in columns_to_subtract_from:
    XS_STRADDLE[col] = XS_STRADDLE[col] - STRADDLE_MERGED['Daily Rate']

### Expectation Hypothesis Test

### Version 1

In [911]:
VIX_TS.reset_index(drop=True, inplace=True)

In [912]:
Y_1 = VIX_TS['1 month'][32:].reset_index(drop=True) - VIX_TS['2 month'][:-32]
Y_2 = VIX_TS['2 month'][32:].reset_index(drop=True) - VIX_TS['3 month'][:-32]
Y_3 = VIX_TS['3 month'][96:].reset_index(drop=True) - VIX_TS['6 month'][:-96]
Y_4 = VIX_TS['6 month'][96:].reset_index(drop=True) - VIX_TS['9 month'][:-96]
Y_5 = VIX_TS['9 month'][96:].reset_index(drop=True) - VIX_TS['12 month'][:-96]

X_1 = (VIX_TS['2 month'][:-32] - VIX_TS['1 month'][:-32]).reset_index(drop=True)
X_2 = ((1/2)*VIX_TS['3 month'][:-32] - VIX_TS['1 month'][:-32]).reset_index(drop=True)
X_3 = (VIX_TS['6 month'][:-96] - VIX_TS['3 month'][:-96]).reset_index(drop=True)
X_4 = ((1/2)*VIX_TS['9 month'][:-96] - VIX_TS['3 month'][:-96]).reset_index(drop=True)
X_5 = ((1/3)*VIX_TS['12 month'][:-96] - VIX_TS['3 month'][:-96]).reset_index(drop=True)

In [913]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-32].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-32].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-96].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-96].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-96].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [914]:
X = df_2[['b_5','c_5']]
Y = Y_5

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables
model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

Regression coefficients (including intercept): const    0.025303
b_5     -0.392307
c_5     -0.757984
dtype: float64
Newey-West standard errors: [0.0018209  0.09448312 0.0789837 ]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.20238229782275052


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


### Version 2

In [915]:
Y_1 = VIX_TS['1 month'][32:].reset_index(drop=True) - VIX_TS['1 month'][:-32]
Y_2 = VIX_TS['2 month'][32:].reset_index(drop=True) - VIX_TS['2 month'][:-32]
Y_3 = VIX_TS['3 month'][96:].reset_index(drop=True) - VIX_TS['3 month'][:-96]
Y_4 = VIX_TS['6 month'][96:].reset_index(drop=True) - VIX_TS['6 month'][:-96]
Y_5 = VIX_TS['9 month'][96:].reset_index(drop=True) - VIX_TS['9 month'][:-96]

X_1 = (VIX_TS['2 month'][:-32] - VIX_TS['1 month'][:-32])+(VIX_TS['2 month'][:-32] - VIX_TS['1 month'][:-32])
X_2 = (VIX_TS['3 month'][:-32] - VIX_TS['2 month'][:-32])+((1/2)*VIX_TS['3 month'][:-32] - VIX_TS['1 month'][:-32])
X_3 = (VIX_TS['6 month'][:-96] - VIX_TS['3 month'][:-96])+(VIX_TS['6 month'][:-96] - VIX_TS['3 month'][:-96])
X_4 = (VIX_TS['9 month'][:-96] - VIX_TS['6 month'][:-96])+((1/2)*VIX_TS['9 month'][:-96] - VIX_TS['3 month'][:-96])
X_5 = (VIX_TS['12 month'][:-96] - VIX_TS['9 month'][:-96])+((1/3)*VIX_TS['12 month'][:-96] - VIX_TS['3 month'][:-96])

In [916]:
data_1 = {'b_1': X_1,
        'b_2': X_2,
        'c_1': VIX_TS['1 month'][:-32].reset_index(drop=True),
       'c_2': VIX_TS['2 month'][:-32].reset_index(drop=True)}

data_2 = {'b_3': X_3,
        'b_4': X_4,
        'b_5': X_5,
       'c_3': VIX_TS['3 month'][:-96].reset_index(drop=True),
       'c_4': VIX_TS['6 month'][:-96].reset_index(drop=True),
       'c_5': VIX_TS['9 month'][:-96].reset_index(drop=True),}

df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

In [917]:
X = df_1[['b_1','c_1']]
Y = Y_1

X = sm.add_constant(X)  # Add a constant (intercept) to the independent variables

model = sm.OLS(Y, X).fit()

# Calculate Newey West

cov_mat = stats.sandwich_covariance.cov_hac(model)
newey_west_se = np.sqrt(np.diag(cov_mat))

# Get the regression coefficients, including the intercept
coefficients = model.params

# Calculate p-values for the expectation hypothesis (b=1)
t_values = (coefficients[1]-1) / newey_west_se[1]

#Calculate p-values
p_values = 2 * scipy.stats.t.sf(abs(t_values), model.df_resid)

# Calculate R-squared
r_squared = model.rsquared

# Output the results
print("Regression coefficients (including intercept):", coefficients)
print("Newey-West standard errors:", newey_west_se)
print("P-values for the expectation hypothesis (b=1):", p_values.round(3))
print("R-squared:", r_squared)

Regression coefficients (including intercept): const    0.016783
b_1     -0.027150
c_1     -0.355029
dtype: float64
Newey-West standard errors: [0.00299867 0.1982755  0.06284436]
P-values for the expectation hypothesis (b=1): 0.0
R-squared: 0.1746571207806864


/Users/juliusraschke/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


### Converting daily to monthly data

In [943]:
cr_VAR_SWAPS = ((XS_VAR_SWAPS)+1).rolling(32).apply(np.prod)-1
cr_VAR_SWAPS[:-31:] = cr_VAR_SWAPS[31::]
cr_VAR_SWAPS = cr_VAR_SWAPS[:-31:]

cr_VIX_FUT = ((XS_VIX_FUT)+1).rolling(32).apply(np.prod)-1
cr_VIX_FUT[:-31:] = cr_VIX_FUT[31::]
cr_VIX_FUT = cr_VIX_FUT[:-31:]

cr_STRADDLE = ((XS_STRADDLE)+1).rolling(32).apply(np.prod)-1
cr_STRADDLE[:-31:] = cr_STRADDLE[31::]
cr_STRADDLE = cr_STRADDLE[:-31:]

### Single Factor Test

In [944]:
pca1 = PCA()
scaler = StandardScaler()

### S&P 500 variance Swap Returns

In [976]:
X = VAR_SWAPS_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VS = pd.DataFrame(data=scaled)
scaled_VS[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VS[[0,1,2,3,4,5]]
scaled_VS = scaled_VS.drop([0,1,2,3,4,5],axis=1)
scaled_VS['Slope'] = -scaled_VS['Slope']

In [977]:
X

,1 month,2 month,3 month,6 month,9 month,12 month
DATE,,,,,,
1996-01-05,0.01708,0.01674,0.01661,0.01828,0.02034,0.01993
1996-01-08,0.01610,0.01589,0.01582,0.01739,0.01863,0.01955
1996-01-09,0.02406,0.01904,0.01675,0.01858,0.01955,0.02088
1996-01-10,0.02494,0.02151,0.02006,0.01978,0.02079,0.02184
1996-01-11,0.01975,0.01932,0.01912,0.01829,0.01920,0.02071
...,...,...,...,...,...,...
2019-06-21,0.02352,0.02683,0.02817,0.03088,0.03170,0.03305
2019-06-24,0.02402,0.02642,0.02783,0.03070,0.03137,0.03290
2019-06-25,0.02718,0.02870,0.02980,0.03222,0.03277,0.03402


### Next day

In [948]:
XS_V_S = XS_VAR_SWAPS
XS_V_S.reset_index(drop=False, inplace=True)

In [949]:
X = scaled_VS[:-1]
Y = (XS_V_S['V_S_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [950]:
cr_V_S = cr_VAR_SWAPS
cr_V_S.reset_index(drop=False, inplace=True)

In [951]:
X = scaled_VS[:-32]
Y = (cr_V_S['V_S_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':32})

### VIX Futures Returns

In [952]:
X = VIX_FUT_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_VF = pd.DataFrame(data=scaled)
scaled_VF[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_VF[[0,1,2,3,4,5]]
scaled_VF = scaled_VF.drop([0,1,2,3,4,5],axis=1)
scaled_VF['Slope'] = -scaled_VF['Slope']

### Next day

In [953]:
XS_V_F = XS_VIX_FUT
XS_V_F.reset_index(drop=False, inplace=True)

In [954]:
X = scaled_VF[:-1]
Y = (XS_V_F['V_F_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [955]:
cr_V_F = cr_VIX_FUT
cr_V_F.reset_index(drop=False, inplace=True)

In [956]:
X = scaled_VF[:-32]
Y = (cr_V_F['V_F_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':32})

### Straddle Returns

In [957]:
X = STRADDLE_MERGED.iloc[:,:6]
X_trans = pca1.fit_transform(X)
X_trans = pd.DataFrame(data=X_trans)
scaled = scaler.fit_transform(X_trans)
scaled_S = pd.DataFrame(data=scaled)
scaled_S[['Level','Slope','Curve','PC4','PC5','PC6']] = scaled_S[[0,1,2,3,4,5]]
scaled_S = scaled_S.drop([0,1,2,3,4,5],axis=1)
scaled_S['Slope'] = -scaled_S['Slope']

### Next day

In [958]:
XS_S = XS_STRADDLE
XS_S.reset_index(drop=False, inplace=True)

In [959]:
X = scaled_S[:-1]
Y = (XS_S['S_R 9 month'][1:]*10000).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':0})

### Next month

In [960]:
cr_S = cr_STRADDLE
cr_S.reset_index(drop=False, inplace=True)

In [961]:
X = scaled_S[:-32]
Y = (cr_S['S_R 1 month'][1:]*100).reset_index(drop=True)

model = sm.OLS(Y, X).fit(cov_type='HAC',cov_kwds={'maxlags':32})

### Economic Significance of Slope as a predictor

In [972]:
scaled_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
ES_VS = pd.concat([XS_VAR_SWAPS[1:].reset_index(drop=True),scaled_VS['Slope_Quantile'][:-1]],axis=1)
VS_Q_Ret = ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(VS_Q_Ret.min() - VS_Q_Ret.max()).T
range_row.index = ['Range']
VS_Q_Ret = pd.concat([VS_Q_Ret, range_row])
VS_Q_Ret

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
0,-0.572038,0.244345,0.386122,0.258562,0.221150,0.182670
1,-1.399156,-0.728917,-0.424027,-0.191416,-0.120554,-0.085899
2,-1.521182,-0.828034,-0.585539,-0.306919,-0.206864,-0.105189
3,-1.490982,-0.903713,-0.533069,-0.376865,-0.181283,-0.023935
4,-2.005382,-1.346360,-0.987533,-0.614128,-0.502718,-0.495175
Range,-1.433343,-1.590706,-1.373655,-0.872690,-0.723868,-0.677846


In [973]:
scaled_VF['Slope_Quantile'] = pd.qcut(scaled_VF['Slope'], q=5, labels=False)
ES_VF = pd.concat([XS_VIX_FUT[1:].reset_index(drop=True),scaled_VF['Slope_Quantile'][:-1]],axis=1)
VF_Q_Ret = ES_VF.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(VF_Q_Ret.min() - VF_Q_Ret.max()).T
range_row.index = ['Range']
VF_Q_Ret = pd.concat([VF_Q_Ret, range_row])
VF_Q_Ret

,V_F_R 1 month,V_F_R 2 month,V_F_R 3 month,V_F_R 6 month,V_F_R 9 month,V_F_R 12 month
0,0.056242,0.027001,0.000456,0.094582,0.108885,0.027323
1,-0.262411,-0.243983,-0.257679,-0.207549,-0.179378,-0.211195
2,-0.201813,-0.223828,-0.208229,-0.162340,-0.138415,-0.050902
3,-0.140691,-0.127868,-0.087659,-0.008243,-0.027570,-0.095389
4,-0.506191,-0.524580,-0.326711,-0.337252,-0.284948,-0.227393
Range,-0.562433,-0.551580,-0.327167,-0.431833,-0.393833,-0.254716


In [974]:
scaled_S['Slope_Quantile'] = pd.qcut(scaled_S['Slope'], q=5, labels=False)
ES_STRADDLE = pd.concat([XS_STRADDLE[1:].reset_index(drop=True),scaled_S['Slope_Quantile'][:-1]],axis=1)
S_Q_Ret = ES_STRADDLE.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(S_Q_Ret.min() - S_Q_Ret.max()).T
range_row.index = ['Range']
S_Q_Ret = pd.concat([S_Q_Ret, range_row])
S_Q_Ret

,S_R 1 month,S_R 2 month,S_R 3 month,S_R 6 month,S_R 9 month,S_R 12 month
0,0.305044,0.296308,0.352745,0.302683,0.249758,0.212182
1,-0.371854,-0.220060,-0.161207,-0.101416,-0.052046,-0.024807
2,-0.420836,-0.268379,-0.176329,-0.104973,-0.071581,-0.047470
3,-0.472459,-0.311763,-0.225431,-0.101285,-0.075660,-0.048131
4,-0.653321,-0.433354,-0.342363,-0.205976,-0.155590,-0.094904
Range,-0.958364,-0.729663,-0.695108,-0.508659,-0.405348,-0.307086


### Robustness of Slope as a predictor

In [1013]:
VS_Robustness = VS_Q_Ret.iloc[5:,:]
VS_Robustness = pd.concat([VS_Robustness, m_VS_Q_Ret.iloc[5:,:]])


In [1014]:
VS_Robustness

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
Range,-1.433343,-1.590706,-1.373655,-0.87269,-0.723868,-0.677846
Range,-54.882620,-39.475750,-35.209378,-24.10614,-20.261915,-14.686826


In [984]:
scaled_VS['Slope_Quantile'] = pd.qcut(scaled_VS['Slope'], q=5, labels=False)
m_ES_VS = pd.concat([cr_VAR_SWAPS,scaled_VS['Slope_Quantile'][:-31]],axis=1)
m_VS_Q_Ret = m_ES_VS.sort_values(by='Slope_Quantile').groupby('Slope_Quantile').mean(numeric_only=True)*100
range_row = pd.DataFrame(m_VS_Q_Ret.min() - m_VS_Q_Ret.max()).T
range_row.index = ['Next-Month Returns']
m_VS_Q_Ret = pd.concat([m_VS_Q_Ret, range_row])
m_VS_Q_Ret

,V_S_R 1 month,V_S_R 2 month,V_S_R 3 month,V_S_R 6 month,V_S_R 9 month,V_S_R 12 month
0,-10.995877,5.646173,9.800210,7.254745,7.167821,5.605746
1,-34.106824,-20.213368,-12.266138,-7.096958,-4.171805,-3.199323
2,-22.775640,-15.977752,-11.104585,-8.190819,-5.502410,-4.158440
3,7.836555,-9.748080,-11.788836,-11.239984,-8.491511,-5.397245
4,-47.046065,-33.829577,-25.409167,-16.851395,-13.094094,-9.081081
Range,-54.882620,-39.475750,-35.209378,-24.106140,-20.261915,-14.686826
